In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

data = pd.read_csv('products.csv')
data = data.iloc[:, 1:]
data.head()

,class0,price,rank1_list,stars,reviews,rank_big,brand,stock,rank1_list_cut,reviews_cut,price_cut,stars_cut,rank_big_cut,rank1_list_cut_n,reviews_cut_n,price_cut_n,stars_cut_n,rank_big_cut_n,r_p
0,x1,243.9,1,1.5,609,21111.0,Furbo,In Stock.,<=10,300-1000,>180,1-2,10000-50000,1,3,10,2,5,30
1,x1,55.5,2,1.3,904,6643.0,TOOGE,In Stock.,<=10,300-1000,40-60,1-2,5000-10000,1,3,3,2,4,9
2,x1,34.4,4,4.1,1916,1451.0,WOPET,In Stock.,<=10,1500-2000,20-40,4-5,1000-5000,1,5,2,5,3,10
3,x1,124.6,5,4.7,844,29857.0,VINSION,In Stock.,<=10,300-1000,120-140,4-5,10000-50000,1,3,7,5,5,21
4,x1,38.4,6,2.5,94,4090.0,TOOGE,In Stock.,<=10,50-100,20-40,2-3,1000-5000,1,1,2,3,3,2


### review量top50的中位数排名 

In [2]:
n = data['class0'].nunique()
df_final = data.sort_values(['class0','reviews'], ascending=[True, False]).groupby('class0', group_keys=False).apply(lambda x: x.head(50))
df_final = df_final.groupby(by='class0', as_index=False)['reviews'].median()
df_final['reviews'] = list(n-np.argsort(df_final['reviews']))
df_final = df_final.rename(columns = {'reviews':'reviews-median_top_50'})
df_final

,class0,reviews-median_top_50
0,x1,9
1,x10,4
2,x2,5
3,x3,8
4,x4,6
5,x5,7
6,x7,2
7,x8,3
8,x9,1


### review区间最多的区间

In [3]:
df1 = data.groupby(by='class0', as_index=False)['reviews_cut_n'].value_counts()
df_final['reviews_cut_n_most_top_10'] = 0*n
for i in range(n):
    df_final['reviews_cut_n_most_top_10'][i] = list(df1[df1['class0']==df_final['class0'][i]]['reviews_cut_n'])[0]
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10
0,x1,9,3
1,x10,4,6
2,x2,5,6
3,x3,8,6
4,x4,6,6
5,x5,7,3
6,x7,2,6
7,x8,3,6
8,x9,1,6


### 品牌数量

In [4]:
df_final['brand_count'] = data.groupby(by='class0', as_index=False)['brand'].nunique()['brand']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count
0,x1,9,3,30
1,x10,4,6,23
2,x2,5,6,26
3,x3,8,6,15
4,x4,6,6,36
5,x5,7,3,20
6,x7,2,6,32
7,x8,3,6,22
8,x9,1,6,17


### 数量最多的品牌产品数量占比

In [5]:
df2 = data.groupby(by=['class0','brand'], as_index=False)['price'].nunique()
df2 = pd.merge(df2, df2.groupby(by='class0', as_index=False)['price'].sum(), how='left', on='class0')
df2['ratio'] = df2['price_x']/df2['price_y']
df_final['brand_top_ratio'] = df2.groupby(by='class0', as_index=False)['ratio'].max()['ratio']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio
0,x1,9,3,30,0.083333
1,x10,4,6,23,0.239130
2,x2,5,6,26,0.183673
3,x3,8,6,15,0.250000
4,x4,6,6,36,0.133333
5,x5,7,3,20,0.319149
6,x7,2,6,32,0.204082
7,x8,3,6,22,0.146341
8,x9,1,6,17,0.360000


### 产品大类排名区间最多的区间

In [6]:
df3 = data.groupby(by='class0', as_index=False)['rank_big_cut_n'].value_counts()
df_final['rank_big_cut_n_most_top_10'] = 0*n
for i in range(n):
    df_final['rank_big_cut_n_most_top_10'][i] = list(df3[df3['class0']==df_final['class0'][i]]['rank_big_cut_n'])[0]
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10
0,x1,9,3,30,0.083333,5
1,x10,4,6,23,0.239130,5
2,x2,5,6,26,0.183673,5
3,x3,8,6,15,0.250000,5
4,x4,6,6,36,0.133333,5
5,x5,7,3,20,0.319149,5
6,x7,2,6,32,0.204082,5
7,x8,3,6,22,0.146341,5
8,x9,1,6,17,0.360000,5


### top50的产品大类排名80%的排名

In [7]:
df4 = data.sort_values(['class0','rank_big'], ascending=[True, True]).groupby('class0', group_keys=False)\
      .apply(lambda x: x.head(int(len(x)*8/10)))
df_final['rank_big_80_top_50'] = df4.groupby(by='class0', as_index=False)['rank_big'].max()['rank_big']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50
0,x1,9,3,30,0.083333,5,21059.0
1,x10,4,6,23,0.239130,5,25908.0
2,x2,5,6,26,0.183673,5,23607.0
3,x3,8,6,15,0.250000,5,22532.0
4,x4,6,6,36,0.133333,5,25643.0
5,x5,7,3,20,0.319149,5,26413.0
6,x7,2,6,32,0.204082,5,19706.0
7,x8,3,6,22,0.146341,5,22325.0
8,x9,1,6,17,0.360000,5,24094.0


### stars区间最多的区间

In [8]:
df5 = data.groupby(by='class0', as_index=False)['stars_cut_n'].value_counts()
df_final['stars_cut_n_most_top_10'] = 0*n
for i in range(n):
    df_final['stars_cut_n_most_top_10'][i] = list(df5[df5['class0']==df_final['class0'][i]]['stars_cut_n'])[0]
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10
0,x1,9,3,30,0.083333,5,21059.0,1
1,x10,4,6,23,0.239130,5,25908.0,4
2,x2,5,6,26,0.183673,5,23607.0,4
3,x3,8,6,15,0.250000,5,22532.0,3
4,x4,6,6,36,0.133333,5,25643.0,2
5,x5,7,3,20,0.319149,5,26413.0,5
6,x7,2,6,32,0.204082,5,19706.0,1
7,x8,3,6,22,0.146341,5,22325.0,2
8,x9,1,6,17,0.360000,5,24094.0,1


### top50stars的平均数

In [9]:
df6 = data.sort_values(['class0','stars'], ascending=[True, False]).groupby('class0', group_keys=False).apply(lambda x: x.head(50))
df_final['stars-mean_top_50'] = df6.groupby(by='class0', as_index=False)['stars'].mean()['stars']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10,stars-mean_top_50
0,x1,9,3,30,0.083333,5,21059.0,1,2.193750
1,x10,4,6,23,0.239130,5,25908.0,4,2.413043
2,x2,5,6,26,0.183673,5,23607.0,4,2.677551
3,x3,8,6,15,0.250000,5,22532.0,3,2.370455
4,x4,6,6,36,0.133333,5,25643.0,2,2.608889
5,x5,7,3,20,0.319149,5,26413.0,5,2.895745
6,x7,2,6,32,0.204082,5,19706.0,1,2.371429
7,x8,3,6,22,0.146341,5,22325.0,2,2.353659
8,x9,1,6,17,0.360000,5,24094.0,1,2.578000


### r_p

In [10]:
df_final['r_p'] = data.groupby(by='class0', as_index=False)['r_p'].mean()['r_p']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10,stars-mean_top_50,r_p
0,x1,9,3,30,0.083333,5,21059.0,1,2.193750,31.291667
1,x10,4,6,23,0.239130,5,25908.0,4,2.413043,36.173913
2,x2,5,6,26,0.183673,5,23607.0,4,2.677551,33.938776
3,x3,8,6,15,0.250000,5,22532.0,3,2.370455,34.704545
4,x4,6,6,36,0.133333,5,25643.0,2,2.608889,32.755556
5,x5,7,3,20,0.319149,5,26413.0,5,2.895745,33.276596
6,x7,2,6,32,0.204082,5,19706.0,1,2.371429,38.632653
7,x8,3,6,22,0.146341,5,22325.0,2,2.353659,38.073171
8,x9,1,6,17,0.360000,5,24094.0,1,2.578000,38.780000


### top50 price的中位数

In [11]:
df7 = data.sort_values(['class0','price'], ascending=[True, False]).groupby('class0', group_keys=False).apply(lambda x: x.head(50))
df_final['price-median_top_50'] = df7.groupby(by='class0', as_index=False)['price'].median()['price']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10,stars-mean_top_50,r_p,price-median_top_50
0,x1,9,3,30,0.083333,5,21059.0,1,2.193750,31.291667,186.75
1,x10,4,6,23,0.239130,5,25908.0,4,2.413043,36.173913,251.25
2,x2,5,6,26,0.183673,5,23607.0,4,2.677551,33.938776,229.70
3,x3,8,6,15,0.250000,5,22532.0,3,2.370455,34.704545,206.70
4,x4,6,6,36,0.133333,5,25643.0,2,2.608889,32.755556,207.50
5,x5,7,3,20,0.319149,5,26413.0,5,2.895745,33.276596,236.30
6,x7,2,6,32,0.204082,5,19706.0,1,2.371429,38.632653,178.40
7,x8,3,6,22,0.146341,5,22325.0,2,2.353659,38.073171,233.80
8,x9,1,6,17,0.360000,5,24094.0,1,2.578000,38.780000,217.40


### price区间最多的区间

In [12]:
df8 = data.groupby(by='class0', as_index=False)['price_cut_n'].value_counts()
df_final['price_cut_n_most_top_10'] = 0*n
for i in range(n):
    df_final['price_cut_n_most_top_10'][i] = list(df8[df8['class0']==df_final['class0'][i]]['price_cut_n'])[0]
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10,stars-mean_top_50,r_p,price-median_top_50,price_cut_n_most_top_10
0,x1,9,3,30,0.083333,5,21059.0,1,2.193750,31.291667,186.75,10
1,x10,4,6,23,0.239130,5,25908.0,4,2.413043,36.173913,251.25,10
2,x2,5,6,26,0.183673,5,23607.0,4,2.677551,33.938776,229.70,10
3,x3,8,6,15,0.250000,5,22532.0,3,2.370455,34.704545,206.70,10
4,x4,6,6,36,0.133333,5,25643.0,2,2.608889,32.755556,207.50,10
5,x5,7,3,20,0.319149,5,26413.0,5,2.895745,33.276596,236.30,10
6,x7,2,6,32,0.204082,5,19706.0,1,2.371429,38.632653,178.40,10
7,x8,3,6,22,0.146341,5,22325.0,2,2.353659,38.073171,233.80,10
8,x9,1,6,17,0.360000,5,24094.0,1,2.578000,38.780000,217.40,10


### top10的价格80%的价格

In [13]:
df9 = data.sort_values(['class0','price'], ascending=[True, False]).groupby('class0', group_keys=False)\
      .apply(lambda x: x.head(8))
df_final['price_80_top_10'] = df9.groupby(by='class0', as_index=False)['price'].min()['price']
df_final

,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10,stars-mean_top_50,r_p,price-median_top_50,price_cut_n_most_top_10,price_80_top_10
0,x1,9,3,30,0.083333,5,21059.0,1,2.193750,31.291667,186.75,10,340.4
1,x10,4,6,23,0.239130,5,25908.0,4,2.413043,36.173913,251.25,10,361.1
2,x2,5,6,26,0.183673,5,23607.0,4,2.677551,33.938776,229.70,10,336.3
3,x3,8,6,15,0.250000,5,22532.0,3,2.370455,34.704545,206.70,10,316.3
4,x4,6,6,36,0.133333,5,25643.0,2,2.608889,32.755556,207.50,10,348.3
5,x5,7,3,20,0.319149,5,26413.0,5,2.895745,33.276596,236.30,10,327.9
6,x7,2,6,32,0.204082,5,19706.0,1,2.371429,38.632653,178.40,10,331.9
7,x8,3,6,22,0.146341,5,22325.0,2,2.353659,38.073171,233.80,10,322.7
8,x9,1,6,17,0.360000,5,24094.0,1,2.578000,38.780000,217.40,10,330.7


In [14]:
df_final.to_csv('df_final.csv')
pd.read_csv('df_final.csv').head()

,Unnamed: 0,class0,reviews-median_top_50,reviews_cut_n_most_top_10,brand_count,brand_top_ratio,rank_big_cut_n_most_top_10,rank_big_80_top_50,stars_cut_n_most_top_10,stars-mean_top_50,r_p,price-median_top_50,price_cut_n_most_top_10,price_80_top_10
0,0,x1,9,3,30,0.083333,5,21059.0,1,2.193750,31.291667,186.75,10,340.4
1,1,x10,4,6,23,0.239130,5,25908.0,4,2.413043,36.173913,251.25,10,361.1
2,2,x2,5,6,26,0.183673,5,23607.0,4,2.677551,33.938776,229.70,10,336.3
3,3,x3,8,6,15,0.250000,5,22532.0,3,2.370455,34.704545,206.70,10,316.3
4,4,x4,6,6,36,0.133333,5,25643.0,2,2.608889,32.755556,207.50,10,348.3
